In [1]:
import re
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import gensim
from sklearn.model_selection import train_test_split
import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt 
import tensorflow as tf
import keras
import numpy as np
import pandas as pd

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import PorterStemmer
st = PorterStemmer()

print('Done')

Done


In [2]:
pd.set_option('display.max_colwidth', None)

rv = pd.read_csv('product_reviews.csv')
rv = rv.head(8)
rv = rv.tail(1)

# Break each review into sentences

In [3]:
# Break review into sentences
from nltk.tokenize import sent_tokenize
st = []
for sentence in rv.content:
    sent_tokenize(sentence)
    st.append(sent_tokenize(sentence))

In [4]:
rv['raw_sentence'] = st
rv.head(3)

,Unnamed: 0,cluster,product_name,price,rating,content,raw_sentence
7,81357,3,JLab Audio Crasher Micro Wireless Bluetooth Speaker | Bluetooth 2.1 | 10 Hour Battery Life | Water Resistant & Dust Resistant | USB Charging | Blue,9.99,5,"I've tried a few of these portable bluetooth speakers. This one is the most portable, easiest to pair up, and the loudest for its size. Also it holds a charge for a good long time.","[I've tried a few of these portable bluetooth speakers., This one is the most portable, easiest to pair up, and the loudest for its size., Also it holds a charge for a good long time.]"


In [5]:
rv1 = rv[['cluster', 'product_name', 'price', 'raw_sentence']]
rv1.head(3)

,cluster,product_name,price,raw_sentence
7,3,JLab Audio Crasher Micro Wireless Bluetooth Speaker | Bluetooth 2.1 | 10 Hour Battery Life | Water Resistant & Dust Resistant | USB Charging | Blue,9.99,"[I've tried a few of these portable bluetooth speakers., This one is the most portable, easiest to pair up, and the loudest for its size., Also it holds a charge for a good long time.]"


In [6]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [7]:
# Create rows for each sentence
rv2 = explode(rv1, 'raw_sentence', fill_value='')

In [8]:
# Convert sentences into lowercase
rv2['sentence'] = [x.lower() for x in rv2.raw_sentence]
rv3 = rv2.drop(columns='raw_sentence')
rv3

,cluster,price,product_name,sentence
0,3,9.99,JLab Audio Crasher Micro Wireless Bluetooth Speaker | Bluetooth 2.1 | 10 Hour Battery Life | Water Resistant & Dust Resistant | USB Charging | Blue,i've tried a few of these portable bluetooth speakers.
1,3,9.99,JLab Audio Crasher Micro Wireless Bluetooth Speaker | Bluetooth 2.1 | 10 Hour Battery Life | Water Resistant & Dust Resistant | USB Charging | Blue,"this one is the most portable, easiest to pair up, and the loudest for its size."
2,3,9.99,JLab Audio Crasher Micro Wireless Bluetooth Speaker | Bluetooth 2.1 | 10 Hour Battery Life | Water Resistant & Dust Resistant | USB Charging | Blue,also it holds a charge for a good long time.


# POS tagging

In [9]:
# POS tagging
pos = []
for sentence in rv3.sentence:
    w = word_tokenize(sentence)
    pos.append(nltk.pos_tag(w))

In [10]:
pos[:1]

[[('i', 'NN'),
  ("'ve", 'VBP'),
  ('tried', 'VBN'),
  ('a', 'DT'),
  ('few', 'JJ'),
  ('of', 'IN'),
  ('these', 'DT'),
  ('portable', 'JJ'),
  ('bluetooth', 'NN'),
  ('speakers', 'NNS'),
  ('.', '.')]]

# Pairing

In [11]:
# Create lists for features and sentiments
ft = [[j[0] for j in i if (j[-1]=="NN" or j[-1]=="NNS" or j[-1]=="NNP" or j[-1]=="NNPS")] for i in pos]
st = [[j[0] for j in i if (j[-1]=="JJ" or j[-1]=="JJR" or j[-1]=="JJS")] for i in pos]

In [12]:
# Lemmatize the lists
ft1 = []
for k in range(len(ft)):
    words=ft[k]
    WNlemma = nltk.WordNetLemmatizer()
    refined_list = [WNlemma.lemmatize(t, pos='n') for t in words]
    ft1.append(refined_list)
    
st1 = []
for k in range(len(st)):
    words=st[k]
    WNlemma = nltk.WordNetLemmatizer()
    refined_list = [WNlemma.lemmatize(t, pos='a') for t in words]
    st1.append(refined_list)

In [13]:
print(ft)
print(ft1)

[['i', 'bluetooth', 'speakers'], ['size'], ['charge', 'time']]
[['i', 'bluetooth', 'speaker'], ['size'], ['charge', 'time']]


In [14]:
print(st)
print(st1)

[['few', 'portable'], ['portable', 'easiest', 'loudest'], ['good', 'long']]
[['few', 'portable'], ['portable', 'easy', 'loud'], ['good', 'long']]


In [15]:
# Create feature-sentiment pairs

pairs = []

for k in range(len(st1)):
    list = [i+"/"+str(j) for i in st1[k] for j in ft1[k]]
    pairs.append(list)

print(len(pairs))
pairs

3


[['few/i',
  'few/bluetooth',
  'few/speaker',
  'portable/i',
  'portable/bluetooth',
  'portable/speaker'],
 ['portable/size', 'easy/size', 'loud/size'],
 ['good/charge', 'good/time', 'long/charge', 'long/time']]